In [4]:
import os
from helpers.utils import parse_dataset
import pandas as pd

DATA_PATH = os.path.join("data", "evals.csv")
GAMES_TO_LOAD = 32_000
parse_dataset(GAMES_TO_LOAD, DATA_PATH)

0.0%
10.0%
20.0%
30.0%
40.0%
50.0%
60.0%
70.0%
80.0%
90.0%
Done in 0.44s


In [5]:
import torch
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision.transforms import Compose
from helpers.fen import fen_to_bitboard
from helpers.data import prepare_chess_frame

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")


def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        pred = model(X)
        loss = loss_fn(pred, y)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            # print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
    return loss


class FenToBits(object):
    def __init__(self, merge_colors: bool):
        self.merge_colors = merge_colors

    def __call__(self, sample):
        return {
            "eval": sample["eval"],
            "board": fen_to_bitboard(sample["fen"], self.merge_colors),
        }


class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""

    def __call__(self, sample):
        board, eval = sample["board"], sample["eval"]
        return {
            "board": torch.from_numpy(board).float(),
            "eval": torch.tensor([eval]).float(),
        }


class ToTuple(object):
    def __call__(self, sample):
        return sample["board"], sample["eval"]


class NeuralNetwork(nn.Module):
    def __init__(self, trial, input_shape):
        super().__init__()

        activation_f = trial.suggest_categorical("activation_func", ["sigmoid", "relu", "tanh"])
        if activation_f == "sigmoid":
            activation_f = nn.Sigmoid
        elif activation_f == "relu":
            activation_f = nn.ReLU
        elif activation_f == "tanh":
            activation_f = nn.Tanh

        conv_layers = trial.suggest_int("conv_layers", 1, 3)
        stack = []
        in_size = input_shape[0]
        kernel = 3
        for i in range(conv_layers):
            out_size = trial.suggest_int(f"conv_out_{i+1}", 2, 128)
            l = nn.Conv2d(in_size, out_size, kernel_size=kernel)
            in_size = out_size
            stack.extend([l, activation_f()])
        dense_layers = trial.suggest_int("dense_layers", 1, 3)
        stack.append(nn.Flatten())
        in_size *= (8 - conv_layers * 2) ** 2
        for i in range(dense_layers):
            out_size = trial.suggest_int(f"dense_out_{i+1}", 4, 128)
            l = nn.Linear(in_size, out_size)
            in_size = out_size
            stack.extend([l, activation_f()])
        stack.append(nn.Linear(in_size, 1))
        self.stack = nn.Sequential(*stack)

    def forward(self, x):
        logits = self.stack(x)
        return logits


class ChessDataset(Dataset):
    def __init__(self, csv_file, transform=None):
        df = pd.read_csv(csv_file)
        self.df = prepare_chess_frame(df, normalize=True)
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        row = self.df.iloc[idx]
        sample = {"fen": row["fen"], "eval": row["eval"]}

        if self.transform:
            sample = self.transform(sample)

        return sample

Using cuda device


In [6]:
import optuna
from optuna.trial import TrialState

def objective(trial):
    BATCH_SIZE = 256
    print(f"Running trial {trial.number}")

    merge_colors = trial.suggest_categorical("merge_colors", ["True", "False"])
    chess_dataset = ChessDataset(
        csv_file=DATA_PATH,
        transform=Compose([FenToBits(merge_colors == "True"), ToTensor(), ToTuple()]),
    )
    train_set, test_set = random_split(chess_dataset, [0.8, 0.2])
    train_dataloader = DataLoader(train_set, batch_size=BATCH_SIZE)
    test_dataloader = DataLoader(test_set, batch_size=BATCH_SIZE)

    dataset_shape = chess_dataset[0][0].shape
    model = NeuralNetwork(trial=trial, input_shape=dataset_shape).to(device)
    loss_fn = nn.MSELoss()

    lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"])
    optimizer = getattr(torch.optim, optimizer_name)(model.parameters(), lr=lr)

    history = {"train_loss": [], "test_loss": []}
    epochs = 16
    for epoch in range(epochs):
        # print(f"Epoch {epoch+1}\n---------------------")
        train_loss = train(train_dataloader, model, loss_fn, optimizer)

        # test
        num_batches = len(test_dataloader)
        model.eval()
        test_loss = 0.0
        with torch.no_grad():
            for X, y in test_dataloader:
                X, y = X.to(device), y.to(device)
                pred = model(X)
                test_loss += loss_fn(pred, y).item()

        test_loss /= num_batches
        trial.report(test_loss, epoch)

        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

        history["train_loss"].append(train_loss)
        history["test_loss"].append(test_loss)
    return history['test_loss'][-1]


if __name__ == "__main__":
    study = optuna.create_study(
        direction="minimize",
        storage="sqlite:///data/db2.sqlite3",
        study_name="chess-ai-2",
        load_if_exists=True,
    )
    study.optimize(objective, n_trials=15)

    pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
    complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

    print("Study statistics: ")
    print("  Number of finished trials: ", len(study.trials))
    print("  Number of pruned trials: ", len(pruned_trials))
    print("  Number of complete trials: ", len(complete_trials))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: ", trial.value)

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

[I 2024-01-06 23:13:51,277] Using an existing study with name 'chess-ai-2' instead of creating a new one.


Running trial 6


[I 2024-01-06 23:15:51,142] Trial 6 finished with value: 0.18468296766281128 and parameters: {'merge_colors': 'False', 'activation_func': 'relu', 'conv_layers': 1, 'conv_out_1': 117, 'dense_layers': 3, 'dense_out_1': 122, 'dense_out_2': 117, 'dense_out_3': 106, 'lr': 1.4290781637791235e-05, 'optimizer': 'RMSprop'}. Best is trial 3 with value: 0.14810833364725112.


Running trial 7


[I 2024-01-06 23:17:49,863] Trial 7 finished with value: 0.1638789391517639 and parameters: {'merge_colors': 'False', 'activation_func': 'relu', 'conv_layers': 1, 'conv_out_1': 40, 'dense_layers': 2, 'dense_out_1': 81, 'dense_out_2': 44, 'lr': 0.00018626736048670136, 'optimizer': 'RMSprop'}. Best is trial 3 with value: 0.14810833364725112.


...

Running trial 19


[I 2024-01-06 23:36:19,923] Trial 19 finished with value: 0.1508908811211586 and parameters: {'merge_colors': 'False', 'activation_func': 'tanh', 'conv_layers': 1, 'conv_out_1': 61, 'dense_layers': 1, 'dense_out_1': 47, 'lr': 0.0030274603975878954, 'optimizer': 'Adam'}. Best is trial 13 with value: 0.14080359399318695.


Running trial 20


[I 2024-01-06 23:36:29,189] Trial 20 pruned. 


Study statistics: 
  Number of finished trials:  21
  Number of pruned trials:  6
  Number of complete trials:  15
Best trial:
  Value:  0.14080359399318695
  Params: 
    merge_colors: False
    activation_func: relu
    conv_layers: 1
    conv_out_1: 76
    dense_layers: 1
    dense_out_1: 46
    lr: 0.0012722620713917813
    optimizer: Adam
